In [30]:
import os
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict, OrderedDict
from sklearn.metrics import confusion_matrix, classification_report

import seaborn as sns
sns.set_theme()
import matplotlib.pyplot as plt

import ipywidgets as widgets
from IPython.display import display, Markdown, HTML, clear_output, display_html

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup, DistilBertModel, DistilBertTokenizer

from src.train import Train
from src.config import Config

In [2]:
display(Markdown("<h2>Book Reviews Sentiment Analysis</h2>"))
loading_section       = ["Prepare Model Data"]
sections              = ["Train", "Evaluation"]
conclusion_section    = ["Summary"]

train_sub_section   = ["Ensemble Model", "LSTM", "BERT"]
me_sub_section      = ["Model Performance", "Model Interpretability"]

accordions = OrderedDict()
accordions["** Loading **"] = widgets.Accordion(children=[widgets.Output() for section in loading_section])
[accordions["** Loading **"].set_title(i, section) for i, section in enumerate(loading_section)]

for section in sections:
    if section == "Train":
        accordions[section] = widgets.Accordion(children=[widgets.Output() for sub_section in train_sub_section])
        [accordions[section].set_title(i, sub_section) for i, sub_section in enumerate(train_sub_section)]
    else:
        accordions[section] = widgets.Accordion(children=[widgets.Output() for sub_section in me_sub_section])
        [accordions[section].set_title(i, sub_section) for i, sub_section in enumerate(me_sub_section)]
        
accordions["** Conclusion **"] = widgets.Accordion(children=[widgets.Output() for section in conclusion_section])
[accordions["** Conclusion **"].set_title(i, section) for i, section in enumerate(conclusion_section)]
        
widget_fields = widgets.Tab(children=[accordions[t] for t in accordions])
[widget_fields.set_title(i, sub) for i, sub in enumerate(accordions.keys())]

<h2>Book Reviews Sentiment Analysis</h2>

[None, None, None, None]

In [3]:
widget_fields

In [66]:
train = Train()
self = train

RANDOM_SEED = 42
np.random.seed(train.MODELLING_CONFIG["RANDOM_SEED"])
torch.manual_seed(train.MODELLING_CONFIG["RANDOM_SEED"])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
%matplotlib agg

with accordions["** Loading **"].children[0]:
    clear_output()
    display(Markdown("<h2> Initiating Modelling Data Preparation ... </h2>"))
    train.prepare_model_data()

    display(train.histogram_plot(xvar=train.token_lens, 
                                 xlabel="Token count: Most of the reviews seem to contain less than 128 tokens, to be on the safe side and choose a maximum length of 160."))

## Sentiment Classification with BERT Using HuggingFace

In [6]:
self.logger.info("  create train data loader...")
train_data_loader = self.create_data_loader(self.data["train_df"], self.tokenizer, 
                                            self.MODELLING_CONFIG["MAX_LEN"], self.MODELLING_CONFIG["BATCH_SIZE"])

self.logger.info("  create validation data loader...")
val_data_loader = self.create_data_loader(self.data["val_df"], self.tokenizer, 
                                          self.MODELLING_CONFIG["MAX_LEN"], self.MODELLING_CONFIG["BATCH_SIZE"])

self.logger.info("  create test data loader...")
test_data_loader = self.create_data_loader(self.data["test_df"], self.tokenizer, 
                                           self.MODELLING_CONFIG["MAX_LEN"], self.MODELLING_CONFIG["BATCH_SIZE"])

  create train data loader...
  create validation data loader...
  create test data loader...


In [7]:
start = time.time()

train_data = next(iter(train_data_loader))

self.logger.info("complete creating train data for pytorch modelling using time {:.2f}s".format(time.time()-start))

complete creating train data for pytorch modelling using time 26.76s


def run():
    self.logger.info("Initialize BERT Model:")
    self.bert_model = BertModel.from_pretrained(self.MODELLING_CONFIG["PRE_TRAINED_MODEL_NAME"])
    
    self.logger.info("  acquire the last hidden state of BERT model...")
    self.last_hidden_state = self.bert_model(input_ids=encoding["input_ids"], attention_mask=encoding["attention_mask"])[0]
    
    self.logger.info("  acquire the pooled output of BERT model...")
    self.pooled_output = self.bert_model(input_ids=encoding["input_ids"], attention_mask=encoding["attention_mask"])[1]

In [34]:
sample_txt = "When was I last outside? I am stuck at home for 2 weeks."

encoding = self.tokenizer.encode_plus(
    sample_txt,
    max_length=32,
    truncation=True,
    add_special_tokens=True, # Add '[CLS]' and '[SEP]'
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors="pt",  # Return PyTorch tensors
)

In [41]:
self.bert_model = DistilBertModel.from_pretrained(self.MODELLING_CONFIG["PRE_TRAINED_MODEL_NAME"])

last_hidden_state = self.bert_model(input_ids=encoding["input_ids"], attention_mask=encoding["attention_mask"])
pooled_output = self.bert_model(input_ids=encoding["input_ids"], attention_mask=encoding["attention_mask"])

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
input_ids = train_data["input_ids"].to(device).to("cuda:0")
attention_mask = train_data["attention_mask"].to(device)

print(input_ids.shape) # batch size x seq length
print(attention_mask.shape) # batch size x seq length

torch.Size([16, 160])
torch.Size([16, 160])


In [42]:
self.bert = DistilBertModel.from_pretrained(self.MODELLING_CONFIG["PRE_TRAINED_MODEL_NAME"])

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [47]:
class SentimentClassifier(nn.Module, Config):
    def __init__(self, n_classes):
        super(SentimentClassifier, self).__init__()
        self.bert = DistilBertModel.from_pretrained(self.MODELLING_CONFIG["PRE_TRAINED_MODEL_NAME"])
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
        
    
    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        output = self.drop(pooled_output)
        
        return self.out(output)

- PYTORCH_CUDA_ALLOC_CONF=max_cached_bytes=4000000000,initial_pool_size=400000000,allocator=cuda

In [48]:
model = SentimentClassifier(len(self.MODELLING_CONFIG["CLASS_NAMES"])) 
model = model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [49]:
# F.softmax(model(input_ids, attention_mask), dim=1)

### Training

To reproduce the training procedure from the BERT paper, AdamW optimizer provided by Hugging Face will be used. It corrects weight decay, so it's similar to the original paper. Linear scheduler with no warmup steps is also used:

- Batch size: 16, 32
- Learning rate (Adam): 5e-5, 3e-5, 2e-5
- Number of epochs: 2, 3, 4

In [69]:
EPOCHS = 10

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
scaler = torch.cuda.amp.GradScaler()
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

In [70]:
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model = model.train()
    
    losses = []
    correct_predictions = 0
  
    for d in tqdm(data_loader):
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)

        correct_predictions += torch.sum(preds==targets)
        losses.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / n_examples, np.mean(losses)

In [74]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()

    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)

            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, targets)

            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())

    return correct_predictions.double() / n_examples, np.mean(losses)

In [75]:
%%time

history = defaultdict(list)
best_accuracy = 0

for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    print("-" * 10)

    train_acc, train_loss = train_epoch(model, train_data_loader, loss_fn, optimizer, device, scheduler, len(self.data["train_df"]))
    print(f"Train loss {train_loss} accuracy {train_acc}")

    val_acc, val_loss = eval_model(model, val_data_loader, loss_fn, device, len(self.data["val_df"]))
    print(f"Val loss {val_loss} accuracy {val_acc}")
    print()

    history["train_acc"].append(train_acc)
    history["train_loss"].append(train_loss)
    history["val_acc"].append(val_acc)
    history["val_loss"].append(val_loss)

    if val_acc > best_accuracy:
        torch.save(model.state_dict(), "best_model_state.bin")
        best_accuracy = val_acc

Epoch 1/10
----------


  0%|                                                                                         | 0/9501 [00:23<?, ?it/s]


ValueError: not enough values to unpack (expected 2, got 1)

In [21]:
plt.plot(history["train_acc"], label="train accuracy")
plt.plot(history["val_acc"], label="validation accuracy")

plt.title("Training History")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend()
plt.ylim([0, 1])

(0.0, 1.0)

## Evaluation

In [ ]:
test_acc, _ = eval_model(model, test_data_loader, loss_fn, device, len(self.data["test_df"]))

test_acc.item()

In [ ]:
def get_predictions(model, data_loader):
    model = model.eval()
  
    review_texts = []
    predictions = []
    prediction_probs = []
    real_values = []
    with torch.no_grad():
        for d in data_loader:
            texts = d["review_text"]
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)

            probs = F.softmax(outputs, dim=1)

            review_texts.extend(texts)
            predictions.extend(preds)
            prediction_probs.extend(probs)
            real_values.extend(targets)

    predictions = torch.stack(predictions).cpu()
    prediction_probs = torch.stack(prediction_probs).cpu()
    real_values = torch.stack(real_values).cpu()
    
    return review_texts, predictions, prediction_probs, real_values

In [ ]:
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(model, test_data_loader)

In [ ]:
print(classification_report(y_test, y_pred, target_names=class_names))

In [ ]:
def show_confusion_matrix(confusion_matrix):
    hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
    hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha="right")
    hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha="right")
    plt.ylabel("True Sentiment")
    plt.xlabel("Predicted Sentiment");

    cm = confusion_matrix(y_test, y_pred)
    df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
    show_confusion_matrix(df_cm)

In [ ]:
idx = 2

review_text = y_review_texts[idx]
true_sentiment = y_test[idx]
pred_df = pd.DataFrame({"class_names": class_names, "values": y_pred_probs[idx]})

In [ ]:
print("\n".join(wrap(review_text)))
print()
print(f"True Sentiment: {class_names[true_sentiment]}")

In [ ]:
sns.barplot(x="values", y="class_names", data=pred_df, orient="h")
plt.ylabel("sentiment")
plt.xlabel("probability")
plt.xlim([0, 1])

## Testing on Raw Text

In [ ]:
review_text = "Today I learn something new, this is great!"

encoded_review = tokenizer.encode_plus(
    review_text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors="pt",
)

In [ ]:
input_ids = encoded_review["input_ids"].to(device)
attention_mask = encoded_review["attention_mask"].to(device)

output = model(input_ids, attention_mask)
_, prediction = torch.max(output, dim=1)

print(f"Review text: {review_text}")
print(f"Sentiment  : {class_names[prediction]}")

## Appendix

In [10]:
class ReviewDataset(Dataset):
    def __init__(self, reviews, targets, tokenizer, max_len):
        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    
    def __len__(self):
        return len(self.reviews)
    
    
    def __getitem__(self, item):
        review = str(self.reviews[item])
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(review,
                                              add_special_tokens=True,
                                              max_length=self.max_len,
                                              return_token_type_ids=False,
                                              pad_to_max_length=True,
                                              return_attention_mask=True,
                                              return_tensors="pt",
                                              )

        return {
            "review_text": review,
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "targets": torch.tensor(target, dtype=torch.long)
            }


def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = ReviewDataset(reviews=df["content"].to_numpy(), targets=df["sentiment"].to_numpy(), tokenizer=tokenizer, max_len=max_len)

    return DataLoader(ds, batch_size=batch_size, num_workers=4)